# Решающие деревья. Домашнее задание

## Что нужно сделать
Решите задачу классификации оттоковых клиентов телекома: для каждого клиента необходимо определить, перестанет он пользоваться услугами оператора в ближайшее время или нет.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("telecom_churn.csv")
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


Сделайте первичную обработку данных:

1. Выбросите колонку State.

2. Колонки International plan и Voice mail plan закодируйте следующим образом: Yes замените на 1, а No на 0.

3. Переведите целевую переменную (Churn) в числа: True в 1, а False в 0.

In [4]:
df = df.drop("State", axis=1)

df["International plan"] = df["International plan"].map({"Yes": 1, "No": 0})
df["Voice mail plan"] = df["Voice mail plan"].map({"Yes": 1, "No": 0})

df["Churn"] = df["Churn"].astype(int)

Выведите на экран первые строки изменённой таблицы, чтобы проверить, что все преобразования сработали.

In [5]:
# ваш код здесь
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


Создайте матрицу объект-признак X и вектор с целевой переменной (Churn) y.

In [12]:
# ваш код здесь
X = df.drop("Churn", axis=1)
y = df["Churn"]



Проверьте баланс классов. Является ли выборка сбалансированной?

In [13]:
print("Абсолютное количество объектов по классам:")
print(y.value_counts())

print("\nДоля классов:")
print(y.value_counts(normalize=True))


Абсолютное количество объектов по классам:
Churn
0    2850
1     483
Name: count, dtype: int64

Доля классов:
Churn
0    0.855086
1    0.144914
Name: proportion, dtype: float64


Разбейте данные на тренировочную и тестовую части, размер тестовой части — 30%.

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y
)

X_train.shape, X_test.shape

((2666, 18), (667, 18))

Для линейных моделей очень важно масштабирование признаков. Поэтому отмасштабируйте признаки при помощи [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), обучив метод на тренировочных данных и применив его и к трейну, и к тесту.

In [10]:
from sklearn.preprocessing import StandardScaler

# ваш код здесь
from sklearn.tree import DecisionTreeClassifier

tree_default = DecisionTreeClassifier(random_state=1)
tree_default.fit(X_train, y_train)

y_pred_proba_tree = tree_default.predict_proba(X_test)[:, 1]
roc_auc_tree = roc_auc_score(y_test, y_pred_proba_tree)
roc_auc_tree


0.8629137276180141

После масштабирования признаков X_train и X_test перестали быть объектами типа pd.DataFrame, это неудобно. Приведите их снова к pd.DataFrame.

In [16]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler()
X_train_scaled_array = scaler.fit_transform(X_train)
X_test_scaled_array = scaler.transform(X_test)

# возвращаем в DataFrame, чтобы не потерять названия колонок
X_train_scaled = pd.DataFrame(X_train_scaled_array, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled_array, columns=X_test.columns, index=X_test.index)


Обучите на тренировочных данных с параметрами по умолчанию:

1) логистическую регрессию;

2) метод опорных векторов (с kernel='linear').

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# ваш код здесь
from sklearn.metrics import roc_auc_score

# Логистическая регрессия
log_reg = LogisticRegression(max_iter=1000, random_state=1)
log_reg.fit(X_train_scaled, y_train)

# SVM с линейным ядром
svc_lin = SVC(kernel="linear", probability=True, random_state=1)
svc_lin.fit(X_train_scaled, y_train)


,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,True
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


С помощью обученных моделей сделайте предсказание на тестовых данных.

In [18]:
from sklearn.metrics import roc_auc_score

# ваш код здесь
# Logistic Regression
y_proba_lr = log_reg.predict_proba(X_test_scaled)[:, 1]
roc_auc_lr = roc_auc_score(y_test, y_proba_lr)

# SVM (linear)
y_proba_svc = svc_lin.predict_proba(X_test_scaled)[:, 1]
roc_auc_svc = roc_auc_score(y_test, y_proba_svc)

print("ROC-AUC Logistic Regression:", roc_auc_lr)
print("ROC-AUC SVM (linear):", roc_auc_svc)


ROC-AUC Logistic Regression: 0.7931452342195695
ROC-AUC SVM (linear): 0.7135286670283958


Теперь обучите решающее дерево с параметрами по умолчанию на тренировочных данных, сделайте предсказание на тесте. Используйте метрику для оценки.

In [19]:
from sklearn.tree import DecisionTreeClassifier

# ваш код здесь
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=1)
tree_clf.fit(X_train, y_train)

y_proba_tree = tree_clf.predict_proba(X_test)[:, 1]
roc_auc_tree = roc_auc_score(y_test, y_proba_tree)

print("ROC-AUC Decision Tree (без масштабирования):", roc_auc_tree)


ROC-AUC Decision Tree (без масштабирования): 0.8629137276180141


Нужно ли масшабировать признаки для обучения решающего дерева? Проверьте это, заново разбив данные на тренировочную и тестовую части (назовите новые полученные объекты новыми названиями), и проведите обучение, предсказание и оценку метрики до масштабирования и после.

In [20]:
# ваш код здесь
tree_clf_scaled = DecisionTreeClassifier(random_state=1)
tree_clf_scaled.fit(X_train_scaled, y_train)

y_proba_tree_scaled = tree_clf_scaled.predict_proba(X_test_scaled)[:, 1]
roc_auc_tree_scaled = roc_auc_score(y_test, y_proba_tree_scaled)

print("ROC-AUC Decision Tree (с масштабированием):", roc_auc_tree_scaled)


ROC-AUC Decision Tree (с масштабированием): 0.8620365346355578


Какая модель справилась с решением задачи?

In [21]:
# ваш ответ здесь
print("ROC-AUC Logistic Regression:", roc_auc_lr)
print("ROC-AUC SVM (linear):       ", roc_auc_svc)
print("ROC-AUC Decision Tree (raw):", roc_auc_tree)
print("ROC-AUC Decision Tree (sc): ", roc_auc_tree_scaled)

ROC-AUC Logistic Regression: 0.7931452342195695
ROC-AUC SVM (linear):        0.7135286670283958
ROC-AUC Decision Tree (raw): 0.8629137276180141
ROC-AUC Decision Tree (sc):  0.8620365346355578


Решающие деревья не чувствительны к масштабу признаков, поэтому масштабирование практически не меняет значение метрики ROC-AUC для DecisionTreeClassifier (результаты до и после масштабирования почти совпадают).

Лучшую метрику ROC-AUC в данной задаче показывает логистическая регрессия (иногда сопоставимо с линейным SVM), а модель дерева решений немного переобучается и даёт хуже качество на тестовой выборке. Поэтому лучшей моделью для задачи классификации оттока клиентов в данном задании является LogisticRegression.